Read the message data into 'df' DataFrame, label columns 

In [ ]:
import pandas as pd
import numpy as np

df=pd.read_table("https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv",header=None)
names=['label','messages']
df.columns=names


Convert values in 'label' column to numerical values and store in 'label_num' column for using MultinomialNB algorithm that only takes numerical data

In [16]:
df['label_num'] = df.label.map({'ham':0, 'spam':1})

Original data is split 5 times so every row is once in test data. Results combined according to test_index and prediction obtained. This is compared with output values of original data to get accuracy

Count Vectorizer is called, Firstly the number of columns are decided based on number of disitnct words in the given data. 
Each row corresponds to a given message sentence and each column value will be the number of occurences of that distinct word (column name) in that sentence. This is all handled by CountVectorizer. 
 
CountVectorizer is internal to Jupyter Notebook, no need to import anything

The parameter to initialize CountVectorizer needs to be changed since it does not recognize single words like 'a' as distinct words. For this ' token_pattern=r"(?u)\b\w\w+\b" ' which is a default parameter is updated to ' token_pattern=r"(?u)\b\w+\b" ' which recognizes single words 

The vectorized data is converted to arrays for each message sentence

Pairwise distance between single test instance/row and every row of training data according to euclidean distance is calculated.

Nearest k neighbour's (k training data rows with least euclidean distance from test row) output is considered and the majority's output is predicted output of this test instance

Algorithm is run for 67 k values that is root of training data size

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

from sklearn.metrics import pairwise_distances

acc=[]
for k in range(1,67):
    results=pd.Series()
    for train_index, test_index in kf.split(df):
        XTrain, XTest, YTrain, ytest = df['messages'].iloc[train_index], df['messages'].iloc[test_index], df['label_num'].iloc[train_index], df['label_num'].iloc[test_index]
    
        cv=CountVectorizer(stop_words=None, token_pattern=r"(?u)\b\w+\b")
        x1=cv.fit_transform(XTrain)
        a1=x1.toarray()
        xtrain=pd.DataFrame(a1,index=train_index, columns =cv.get_feature_names())
        x2=cv.transform(XTest)
        a2=x2.toarray()
        xtest=pd.DataFrame(a2, index=test_index, columns =cv.get_feature_names())
  
        Dist=pairwise_distances(xtest, xtrain, metric='euclidean') 
    
        prediction=[]
        for i in range(0,xtest.shape[0]):
            sorting=pd.Series(Dist[i],index=train_index)    
            sorting=sorting.sort_values()
            #print(sorting)
        
            Output_indexes=sorting[0:k].index
        
            Output=YTrain[Output_indexes[0:k]]
            
            O=pd.Series(Output)
        
            prediction.append(max(O.value_counts().index))
    
            indexes=list(test_index)
    
        res=pd.Series(prediction,index=indexes)
        results=results.append(res)     
    
    results=results.sort_index()   
    from sklearn import metrics
    print("Accuracy:",(metrics.accuracy_score(df['label_num'], results)*100),"%",k)
    acc.append(metrics.accuracy_score(df['label_num'], results)*100)
print(max(acc),acc.index(max(acc)))    

Accuracy: 95.04666188083274 % 1
Accuracy: 95.13639626704953 % 2
Accuracy: 95.22613065326632 % 3
Accuracy: 95.20818377602296 % 4
Accuracy: 95.2440775305097 % 5
Accuracy: 95.26202440775306 % 6
Accuracy: 95.2440775305097 % 7
Accuracy: 95.19023689877962 % 8
Accuracy: 95.17229002153626 % 9
Accuracy: 95.1543431442929 % 10
Accuracy: 95.10050251256281 % 11
Accuracy: 94.99282124910266 % 12
Accuracy: 94.93898061737258 % 13
Accuracy: 94.86719310839914 % 14
Accuracy: 94.79540559942569 % 15
Accuracy: 94.7056712132089 % 16
Accuracy: 94.58004307250538 % 17
Accuracy: 94.56209619526203 % 18
Accuracy: 94.52620244077531 % 19
Accuracy: 94.47236180904522 % 20
Accuracy: 94.47236180904522 % 21
Accuracy: 94.23905240488155 % 22
Accuracy: 94.14931801866476 % 23
Accuracy: 94.1313711414214 % 24
Accuracy: 94.11342426417804 % 25
Accuracy: 94.07753050969131 % 26
Accuracy: 94.09547738693468 % 27
Accuracy: 94.11342426417804 % 28
Accuracy: 94.00574300071789 % 29
Accuracy: 93.93395549174444 % 30
Accuracy: 93.79038047379

In [13]:
from sklearn import metrics
print("Accuracy:",(metrics.accuracy_score(df['label_num'], results)*100),"%")  

Accuracy: 95.10050251256281 %
